In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [5]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                 transform=transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize((0.1307,),(0.3081,))
                 ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize((0.1307,),(0.3081,))
                 ])),
    batch_size=batch_size, shuffle=True)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # weight가 있는 것들
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # Feature Extraction
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)

        #Fully Connected (Classification)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model = Net().to(device)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [12]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [14]:
model.train() # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [15]:
data, target = next(iter(train_loader))

In [16]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [17]:
data, target = data.to(device), target.to(device)

In [18]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [19]:
optimizer.zero_grad() # clear

In [20]:
output = model(data)

In [21]:
loss = F.nll_loss(output, target)

In [22]:
loss.backward()

In [23]:
optimizer.step()

In [24]:
# 정리

In [25]:
epochs = 1
log_interval = 100

In [26]:
for epoch in range(1, epochs + 1):
    # train mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # 한 번 학습 완료
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)\tLoss: {:.6f}]'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100 * batch_idx/ len(train_loader), loss.item()
            ))
        # 표시를 위해서 윗부분 작성
        
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)	Loss: 2.290736]
Train Epoch: 1 [6400/60000 (11%)	Loss: 2.228956]
Train Epoch: 1 [12800/60000 (21%)	Loss: 2.121075]
Train Epoch: 1 [19200/60000 (32%)	Loss: 1.893548]
Train Epoch: 1 [25600/60000 (43%)	Loss: 1.570037]
Train Epoch: 1 [32000/60000 (53%)	Loss: 1.213112]
Train Epoch: 1 [38400/60000 (64%)	Loss: 0.993902]
Train Epoch: 1 [44800/60000 (75%)	Loss: 0.778629]
Train Epoch: 1 [51200/60000 (85%)	Loss: 0.732979]
Train Epoch: 1 [57600/60000 (96%)	Loss: 0.624407]


In [28]:
model.eval() # 평가모드

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [31]:
test_loss = 0 # 계산용
correct = 0

with torch.no_grad(): # test 모드일때는 꺼줘야된대
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()
    

In [41]:
output.shape

torch.Size([64, 10])

In [38]:
pred.shape

torch.Size([64, 1])

In [42]:
target.shape

torch.Size([64])

In [39]:
test_loss

33.745365142822266

In [40]:
correct

53

In [43]:
test_loss /= len(test_loader.dataset)

In [44]:
test_loss

0.0033745365142822266

In [47]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


Test set: Average Loss: 0.4799, Accuracy: 8661/10000 (87%)

